In [2]:
import pandas as pd
import numpy as np

#read one month of data
bike_df=pd.read_csv('201907-citibike-tripdata.csv')

In [3]:
#change column names
bike_df.rename(columns={'tripduration':'duration','starttime':'start','stoptime':'stop','start station name':'start_station',
               'start station id':'start_id','start station latitude':'start_lat',
                'start station longitude':'start_long','end station id':'end_id',
               'end station name':'end_station','end station latitude':'end_lat','end station longitude':'end_long',
               'bikeid':'bike','usertype':'user','birth year':'birth_year'},inplace=True)

In [4]:
bike_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2181064 entries, 0 to 2181063
Data columns (total 15 columns):
duration         int64
start            object
stop             object
start_id         float64
start_station    object
start_lat        float64
start_long       float64
end_id           float64
end_station      object
end_lat          float64
end_long         float64
bike             int64
user             object
birth_year       int64
gender           int64
dtypes: float64(6), int64(4), object(5)
memory usage: 249.6+ MB


In [5]:
#change column types
bike_df['start']=pd.to_datetime(bike_df['start'])
bike_df['stop']=pd.to_datetime(bike_df['stop'])
bike_df['start_id'].apply(lambda x: str(int(x)) if not pd.isna(x) else x)
bike_df['end_id'].apply(lambda x: str(int(x)) if not pd.isna(x) else x)
bike_df['bike'].apply(lambda x: str(x) if not pd.isna(x) else x)
bike_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2181064 entries, 0 to 2181063
Data columns (total 15 columns):
duration         int64
start            datetime64[ns]
stop             datetime64[ns]
start_id         float64
start_station    object
start_lat        float64
start_long       float64
end_id           float64
end_station      object
end_lat          float64
end_long         float64
bike             int64
user             object
birth_year       int64
gender           int64
dtypes: datetime64[ns](2), float64(6), int64(4), object(3)
memory usage: 249.6+ MB


In [6]:
#add day_of_week and hour colummns
bike_df['start_day']=bike_df['start'].dt.weekday
bike_df['end_day']=bike_df['stop'].dt.weekday

bike_df['start_hour']=bike_df['start'].dt.hour
bike_df['end_hour']=bike_df['stop'].dt.hour

#change start_day and end_day to weekday names
weekdays=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
bike_df['start_day']=bike_df['start_day'].apply(lambda x: weekdays[x])
bike_df['end_day']=bike_df['end_day'].apply(lambda x: weekdays[x])

I start creating some quick plots. Eventually I will keep only the ones that illustrate some insights about the data. 

At this point I also sometimes choose to do things the long way. So for example I'll avoid using pivot_table() function for some data manipulation even if this function would considerably simplify the code. This is because I like to practice a little more the basic pandas functions. Eventually I will replace the longer code with the shorter versions, so that it's easier to read. 

In [151]:
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, FactorRange,HoverTool,Legend,LegendItem
from math import pi
from bokeh.palettes import small_palettes

output_notebook()

Loading BokehJS ...

In [152]:
def make_hist_df(series,rnge,bins=50,normalized=False):
    #takes a series and range and returns a dataframe that is ready to be loaded in ColumnDataSource and used for histogram plot
    hist,edges=np.histogram(series,bins,rnge)
    #if normalized, return percentage of group total instead of group count
    if normalized:
            return pd.DataFrame({'top':hist/series.count(),'left':edges[:-1],'right':edges[1:]})
    return pd.DataFrame({'top':hist,'left':edges[:-1],'right':edges[1:]})

def plot_histogram(data_df,legend=None,color='blue',bottom=0,alpha=.3):
    #takes a dataframe with tops, left, right and plots it in a histogram
    p.quad(left=data_df['left'], right=data_df['right'], top=data_df['top'], bottom=bottom,
           fill_color=color, line_color=None, alpha=.3, legend=legend)

In [165]:
#filter bike_df by gender
bike_male=bike_df[bike_df['gender']==1]
bike_female=bike_df[bike_df['gender']==2]

#use make_hist_df to prepare dataframes for histogram plotting 
hist_df_1=make_hist_df(bike_male['duration']/60,[0,60],60,True)
hist_df_2=make_hist_df(bike_female['duration']/60,[0,60],60,True)

p=figure(title='Trip Length by Gender',plot_height=400,plot_width=800)

#plot histrograms 
plot_histogram(hist_df_1,'Male','orange')
plot_histogram(hist_df_2,'Female')

show(p)

In [154]:
#same as above, but with user types
#i first filter the data by user type, then i create dataframes and finally i plot them
tourist_df=bike_df[bike_df['user']=='Customer']
local_df=bike_df[bike_df['user']=='Subscriber']

hist_df_1=make_hist_df(tourist_df['duration']/60,[0,60],60,True)
hist_df_2=make_hist_df(local_df['duration']/60,[0,60],60,True)

p=figure(title='Trip Length by User Type',plot_height=400,plot_width=800)

plot_histogram(hist_df_1,'Tourist')
plot_histogram(hist_df_2,'Local','orange')

show(p)    

In [398]:
#create column with age groups
bike_df['age_group']=pd.cut(bike_df['birth_year'],bins=[0,1969,1989,2019],labels=['Old','Middle-Aged','Young'])

In [229]:
young_df=bike_df[bike_df['age_group']=='Young']
middle_df=bike_df[bike_df['age_group']=='Middle-Aged']
old_df=bike_df[bike_df['age_group']=='Old']

hist_df_1=make_hist_df(young_df['duration']/60,[0,60],60,True)
hist_df_2=make_hist_df(middle_df['duration']/60,[0,60],60,True)
hist_df_3=make_hist_df(old_df['duration']/60,[0,60],60,True)

p=figure(title='Trip Length by Age Group',plot_height=400,plot_width=800)
plot_histogram(hist_df_1,'Young')
plot_histogram(hist_df_2,'Middle-Aged','red')
plot_histogram(hist_df_3,'Old','green')
show(p)    

Next I look at how trips are distributed by hour of day and by day of week, relative to gender, user type, and age group. The most appropriate chart for this is a stacked or grouped bar chart, but when dealing with hours of day and only two groups (as in the case of gender and user type) I think overlapping bar charts are more suggestive, so I'll use them instead (I think it makes the comparison between subgroups easier).

In [ ]:
#this cell manipulates the data to create a simple dataframe that can be easily used by bokeh to build a chart.
#this dataframe can practically be done in a few lines with pivot_table() (see below), but here I do it the long way.

#count trips started by men by weekday 
male_by_day=bike_male.groupby('start_day')['start_id'].count()/bike_male['start_id'].count()
male_by_day=male_by_day.to_frame()
male_by_day.rename(columns={'start_id':'male_count'},inplace=True)
male_by_day.reset_index(inplace=True)

#same for women
female_by_day=bike_female.groupby('start_day')['start_id'].count()/bike_female['start_id'].count()
female_by_day=female_by_day.to_frame()
female_by_day.rename(columns={'start_id':'female_count'},inplace=True)
female_by_day.reset_index(inplace=True)

#join the data
weekday_counts_gender_df=pd.merge(male_by_day,female_by_day,on='start_day')

In [404]:
from bokeh.transform import dodge
source=ColumnDataSource(weekday_counts_gender_df)

p=figure(x_range=weekdays,y_range=(0,.3),plot_width=800,plot_height=400)

p.vbar(x=dodge('start_day',-0.2,range=p.x_range),top='male_count',width=.33,source=source,color='blue',legend='Male',alpha=.3)
p.vbar(x=dodge('start_day',0.2,range=p.x_range),top='female_count',width=.33,source=source,color='red',legend='Female',alpha=.3)
p.legend.location = "top_right"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.x_range.range_padding = 0.02

show(p)

In [226]:
#count trips started by customers by weekday in a dataframe
tourist_by_day=tourist_df.groupby('start_day')['start_id'].count()/tourist_df['start_id'].count()
tourist_by_day=tourist_by_day.to_frame()
tourist_by_day.rename(columns={'start_id':'tourist_count'},inplace=True)
tourist_by_day.reset_index(inplace=True)

#same for subscribers
local_by_day=local_df.groupby('start_day')['start_id'].count()/local_df['start_id'].count()
local_by_day=local_by_day.to_frame()
local_by_day.rename(columns={'start_id':'local_count'},inplace=True)
local_by_day.reset_index(inplace=True)

#join the two dataframes
weekday_counts_user_df=pd.merge(tourist_by_day,local_by_day,on='start_day')

In [405]:
source=ColumnDataSource(weekday_counts_user_df)

p=figure(x_range=weekdays,y_range=(0,.3),plot_width=800,plot_height=400)

p.vbar(x=dodge('start_day',-0.2,range=p.x_range),top='tourist_count',width=.33,source=source,color='blue',legend='Customer',
       alpha=.3)
p.vbar(x=dodge('start_day',0.2,range=p.x_range),top='local_count',width=.33,source=source,color='red',legend='Subscriber',
       alpha=.3)
p.legend.location = "top_right"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.x_range.range_padding = 0.02

show(p)

In [299]:
#create dataframe with trips taken each weekday by the three age groups i created
df=bike_df.pivot_table(index='start_day',columns='age_group',values='bike',aggfunc=len)

#normalize the data - instead of count i use percentage of total trips by respective age-group
df['Middle-Aged']=df['Middle-Aged']/df['Middle-Aged'].sum()
df['Young']=df['Young']/df['Young'].sum()
df['Old']=df['Old']/df['Old'].sum()

df.reset_index(inplace=True)
del df.columns.name
weekday_counts_age_group=df

In [300]:
source=ColumnDataSource(weekday_counts_age_group)

p=figure(x_range=weekdays,y_range=(0,.3),plot_width=800,plot_height=400)

p.vbar(x=dodge('start_day',-0.22,range=p.x_range),top='Young',width=.2,source=source,color='blue',legend='Youngsters',alpha=.3)
p.vbar(x=dodge('start_day',0,range=p.x_range),top='Middle-Aged',width=.2,source=source,color='red',legend='Middle Aged',alpha=.3)
p.vbar(x=dodge('start_day',0.22,range=p.x_range),top='Old',width=.2,source=source,color='brown',legend='Elderly',alpha=.3)

p.legend.location = "top_right"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.x_range.range_padding = 0.02

show(p)

In [349]:
df=bike_df.pivot_table(index='start_hour',columns='gender',values='bike',aggfunc=len)
df.reset_index(inplace=True)
del df.columns.name
df.rename(columns={1:'male',2:'female'},inplace=True)
df.drop(columns=0, inplace=True)
df['start_hour']=df['start_hour'].apply(lambda x: str(x))

In [362]:
df['male']=df['male']/df['male'].sum()
df['female']=df['female']/df['female'].sum()

In [371]:
source=ColumnDataSource(df)

p=figure(x_range=df['start_hour'].tolist(),plot_width=800,plot_height=400)

p.vbar(x='start_hour',top='male',width=1,source=source,color='blue',legend='Male',alpha=.3,line_color=None)
p.vbar(x='start_hour',top='female',width=1,source=source,color='orange',legend='Female',alpha=.3,line_color=None)

p.legend.location = "top_left"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.x_range.range_padding = 0.04

show(p)

As we can see, there is not much variation by gender in the time of day trips are started. I looks next at distribution of trips starts relative to user type.

In [373]:
df=bike_df.pivot_table(index='start_hour',columns='user',values='bike',aggfunc=len)
df.reset_index(inplace=True)
del df.columns.name
df['start_hour']=df['start_hour'].apply(lambda x: str(x))

df['Subscriber']=df['Subscriber']/df['Subscriber'].sum()
df['Customer']=df['Customer']/df['Customer'].sum()

In [379]:
source=ColumnDataSource(df)

p=figure(x_range=df['start_hour'].tolist(),plot_width=800,plot_height=400)

p.vbar(x='start_hour',top='Subscriber',width=1,source=source,color='blue',legend='Subscribers',alpha=.3,line_color=None)
p.vbar(x='start_hour',top='Customer',width=1,source=source,color='orange',legend='Customers',alpha=.3,line_color=None)

p.legend.location = "top_left"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.x_range.range_padding = 0.04

show(p)

This is a much more revealing chart. It looks like subscribers use the bikes more for commuting (hence spikes in the morning and early evening) while customers (which, again, are probably mostly tourists but also some occasional New York users) use the bikes more during late mornings and in the afternoons. Customers also use the bikes more during late night, but here the difference from subscribers is not very pronounced. 

Next I draw the same chart using group ages:

In [382]:
df=bike_df.pivot_table(index='start_hour',columns='age_group',values='bike',aggfunc=len)
df.reset_index(inplace=True)
del df.columns.name
df['start_hour']=df['start_hour'].apply(lambda x: str(x))

df['Middle-Aged']=df['Middle-Aged']/df['Middle-Aged'].sum()
df['Old']=df['Old']/df['Old'].sum()
df['Young']=df['Young']/df['Young'].sum()


In [389]:
source=ColumnDataSource(df)

p=figure(x_range=df['start_hour'].tolist(),plot_width=800,plot_height=400)

p.vbar(x=dodge('start_hour',-0.22,range=p.x_range),top='Young',width=.2,source=source,color='blue',legend='Youngsters',alpha=.3,line_color=None)
p.vbar(x=dodge('start_hour',0,range=p.x_range),top='Middle-Aged',width=.2,source=source,color='orange',legend='Middle Aged',alpha=.3,line_color=None)
p.vbar(x=dodge('start_hour',0.22,range=p.x_range),top='Old',width=.2,source=source,color='green',legend='Elderly',alpha=.3,line_color=None)

p.legend.location = "top_left"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.x_range.range_padding = 0.04

show(p)

In [26]:
def create_counts_df(filtered_df):
    #takes a filtered df (ex: male only, or subscribers only), and returns sorted df with station name, lat and long,
    #start_counts,end_counts and total_counts for that station

    #create df with number of starts for each station 
    gb=filtered_df.groupby(['start_station','start_lat','start_long'])['start_station'].count()
    start_counts_df=gb.to_frame()
    start_counts_df.rename(columns={'start_station':'start_counts'},inplace=True)
    start_counts_df.reset_index(inplace=True)

    #create df with number of stops for each station 
    gb=bike_male.groupby('end_station')['end_station'].count()
    end_counts_df=gb.to_frame()
    end_counts_df.rename(columns={'end_station':'end_counts'},inplace=True)
    end_counts_df.reset_index(inplace=True)

    #link the two dataframes
    counts_df=start_counts_df.merge(end_counts_df,left_on='start_station',right_on='end_station')
    counts_df.drop(columns=['end_station'],inplace=True)
    counts_df.rename(columns={'start_station':'station'},inplace=True)
    
    #calculate total and order in descending order
    counts_df['total_counts']=counts_df['start_counts']+counts_df['end_counts']
    counts_df.sort_values('total_counts',axis=0,ascending=False,inplace=True)
    return counts_df

In [27]:
male_counts_df=create_counts_df(bike_male)
male_counts_df

,station,start_lat,start_long,start_counts,end_counts,total_counts
584,Pershing Square North,40.751873,-73.977706,13240,13258,26498
125,8 Ave & W 31 St,40.750585,-73.994685,10208,9904,20112
327,E 17 St & Broadway,40.737050,-73.990093,8664,8659,17323
180,Broadway & E 22 St,40.740343,-73.989551,8569,8706,17275
237,Christopher St & Greenwich St,40.732916,-74.007114,8111,8216,16327
20,12 Ave & W 40 St,40.760875,-74.002777,7652,7975,15627
706,W 21 St & 6 Ave,40.741740,-73.994156,7708,7653,15361
771,West St & Chambers St,40.717548,-74.013221,7418,7671,15089
179,Broadway & E 14 St,40.734546,-73.990741,7375,7191,14566
723,W 41 St & 8 Ave,40.756405,-73.990026,7107,7349,14456
